In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures

## Data

In [ ]:
fish_data = pd.read_csv('./data/Fish.csv')

fish_data.info()

In [ ]:
index = fish_data[ fish_data['Weight'] == 0.0].index
fish_data.drop(index, inplace = True)

fish_data.describe()

In [ ]:
# Separate Target from features
y_fish = fish_data.iloc[:, 1]
X_fish = fish_data.drop(columns=['Weight'])
X_fish = X_fish.drop(columns=['Species'])

# Convert into numpy arrays
X_fish = X_fish.values
y_fish = y_fish.values

# Split into Training and Test Groups
X_fish_train, X_fish_test, y_fish_train, y_fish_test = train_test_split(X_fish, y_fish, test_size = 0.2, random_state = 0)

## Training

In [ ]:
poly = PolynomialFeatures(2)

X_fish_train_pol = poly.fit_transform(X_fish_train)
X_fish_test_pol = poly.fit_transform(X_fish_test)

df_check = pd.DataFrame(X_fish_train_pol)
df_check.head(5)

In [ ]:
regressor = LinearRegression()

regressor.fit(X_fish_train_pol, y_fish_train)

## Evaluation

In [ ]:
y_fish_pol_pred = regressor.predict(X_fish_test_pol)

pred_pol_compare = pd.DataFrame()
pred_pol_compare['Prediction'] = y_fish_pol_pred.tolist()
pred_pol_compare['Actual'] = y_fish_test.tolist()

print(pred_pol_compare)

In [ ]:
mse_pol = mean_squared_error(y_fish_test, y_fish_pol_pred)
r2s_pol = r2_score(y_fish_test, y_fish_pol_pred)

print("MSE: %s | R2S: %s" % (mse_pol, r2s_pol))